In [ ]:
import cv2
import numpy as np
import os


sift = cv2.SIFT_create()


def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors


def match_descriptors(descriptors1, descriptors2):
    if descriptors1 is None or descriptors2 is None:
        return []

    descriptors1 = np.float32(descriptors1)
    descriptors2 = np.float32(descriptors2)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    return matches


def classify_image(test_image_path, dataset_folder):
    test_image = cv2.imread(test_image_path)
    if test_image is None:
        raise ValueError(f"Failed to load test image at {test_image_path}")

    test_keypoints, test_descriptors = extract_features(test_image)
    best_match_count = 0
    best_class = None

    
    for class_name in os.listdir(dataset_folder):
        class_folder = os.path.join(dataset_folder, class_name)

        if os.path.isdir(class_folder):
            total_matches = 0

            
            for image_name in os.listdir(class_folder):
                image_path = os.path.join(class_folder, image_name)
                class_image = cv2.imread(image_path)

                class_keypoints, class_descriptors = extract_features(class_image)

                
                matches = match_descriptors(test_descriptors, class_descriptors)
                total_matches += len(matches)


            if total_matches > best_match_count:
                best_match_count = total_matches
                best_class = class_name

    return best_class, best_match_count


test_image_path = r'C:\Users\student\Desktop\CV20\test2.jpg'
dataset_folder = r'C:\Users\student\Downloads\archive (5)\geometric shapes dataset'


try:
    best_class, match_count = classify_image(test_image_path, dataset_folder)
    print(f"The test image is classified as: {best_class} with {match_count} matches.")
except ValueError as e:
    print(e)


In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

MIN_MATCH_COUNT = 3


img2 = cv2.imread(r"C:\Users\madhu\Downloads\objects.jpeg") 
img1 = cv2.imread(r"C:\Users\madhu\OneDrive\Pictures\Screenshots\Screenshot 2025-03-17 211340.png")  


img_rgb = img2.copy()


alg = cv2.SIFT_create()  


kp1, des1 = alg.detectAndCompute(img1, None)
kp2, des2 = alg.detectAndCompute(img2, None)


x = np.array([kp2[0].pt])

for i in range(len(kp2)):
    x = np.append(x, [kp2[i].pt], axis=0)

x = x[1:len(x)]  


bandwidth = estimate_bandwidth(x, quantile=0.1, n_samples=500)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, cluster_all=True)
ms.fit(x)

labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)

n_clusters_ = len(labels_unique)
print("Number of estimated clusters: %d" % n_clusters_)


s = [None] * n_clusters_
for i in range(n_clusters_):
    l = ms.labels_

    d, = np.where(l == i)

    s[i] = list(kp2[xx] for xx in d)

des2_ = des2


for i in range(n_clusters_):
    kp2 = s[i]

    d, = np.where(labels == i)
    des2 = des2_[d, :]

    if len(kp2) < 2 or len(kp1) < 2:
        continue

    
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    des1 = np.float32(des1)
    des2 = np.float32(des2)

    matches = flann.knnMatch(des1, des2, 2)

    good = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good.append(m)

    
    if len(good) > MIN_MATCH_COUNT:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 2)

        if M is None:
            print("No Homography")
            continue

       
        h, w = img1.shape[:2]
        corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        transformedCorners = cv2.perspectiveTransform(corners, M)

        x = int(transformedCorners[0][0][0])
        y = int(transformedCorners[0][0][1])

        
        cv2.rectangle(img_rgb, (x, y), (x+w, y+h), (0, 0, 255), 3)
        img2 = cv2.polylines(img2, [np.int32(transformedCorners)], True, (0, 0, 255), 2, cv2.LINE_AA)


img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)


plt.figure(figsize=(12, 6))

plt.subplot(121)
plt.title('Template Image')
plt.axis('off')
plt.imshow(img1_rgb)

plt.subplot(122)
plt.title('Scene Image with Detected Object')
plt.axis('off')
plt.imshow(img_rgb)

plt.tight_layout()
plt.show()
